In [2]:
from torchvision import transforms as T
from torch.utils.data import DataLoader
import io
import zipfile
from torchnet import meter
import torch
import os
from PIL import Image
from torch.utils import data
import numpy as np
import pandas as pd
from torchvision import transforms as T
import io
import zipfile

In [197]:
import torch
import os
from PIL import Image
from torch.utils import data
import numpy as np
import pandas as pd
from torchvision import transforms as T
import io
import zipfile

In [198]:
from torch import nn

In [211]:
class CNN_Model(nn.Module):
    def __init__(self):
        super(CNN_Model, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1).cuda()
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1).cuda()
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=64, kernel_size=3, stride=1, padding=1).cuda()
        self.conv4 = nn.Conv1d(in_channels=64, out_channels=32, kernel_size=3, stride=1, padding=1).cuda()
        self.conv5 = nn.Conv1d(in_channels=32, out_channels=16, kernel_size=3, stride=1, padding=1).cuda()
        self.classifier = nn.Sequential(
            nn.Linear(2048, 20),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(20, 1),
            nn.Sigmoid()
        ).cuda()
        # (5120x128 and 160x20)
    def forward(self, x):

        x = torch.relu(self.conv1(x))
        # print(x.shape)
        x = torch.relu(self.conv2(x))
        # print(x.shape)
        x = torch.relu(self.conv3(x))
        # print(x.shape)
        x = torch.relu(self.conv4(x))
        # print(x.shape)
        x = torch.relu(self.conv5(x))
        # print(x.shape)
        x = torch.flatten(x,1)
        x = self.classifier(x)
        
        return x

    def save(self, name=None):
        """
        save the model
        """
        if name is None:
            prefix = 'checkpoints/' + 'face_classifier_'
            name = time.strftime(prefix + '%m%d_%H:%M:%S.pth')
        torch.save(self.state_dict(), name)
        return name

In [1]:



class DEAP(data.Dataset):
    '''
    DEAP dataset for per-subject experiments

    Parameters:
        modal   : data modality; {'face', 'eeg', 'peri', 'bio', 'faceeeg', 'faceperi', 'facebio'}, default = 'facebio'.
        subject : subject ID; an integer between 1 and 22, default = 1.
        k       : kth fold; an integer between 1 and 10, default = 1.
        kind    : dataset type; {'train', 'val', 'all'}, default = 'all'.
        indices : index of data samples (for dataset shuffle); an list of integers, default = list(range(2400)).
        label   : emotion label; {'valence', 'arousal'}, defualt = 'valence'.
    '''
    def __init__(self, modal='face', subject=1, k=1, kind='all', indices=list(range(2400)),label='valence'):
        self.modal = modal
        self.subject = subject
        self.k = k
        self.kind = kind
        self.label = label
        self.bio_path = f'./data/DEAP/bio/s{subject}.zip'
        self.label_path = f'./data/DEAP/labels/'
        self.face_path = f'./data/DEAP/faces/s{subject}.zip'
        self.labels = pd.read_csv(self.label_path+'participant_ratings.csv')
        # print(self.labels)
        # uncomment
        self.bio_zip = zipfile.ZipFile(self.bio_path, 'r')
        # print(self.face_path)
        self.face_zip = zipfile.ZipFile(self.face_path, 'r')
        
        
        self.size = len(indices)

        if kind == 'train':
            self.indices = indices[:int((k - 1) * self.size / 10)] + indices[int(k * self.size / 10):]
        if kind == 'val':
            self.indices = indices[int((k - 1) * self.size / 10):int(k * self.size / 10)]
        if kind == 'all':
            self.indices = indices

    def __getitem__(self, i):
        index = self.indices[i]
        trial = index // 60 + 1
        segment = index % 60 + 1
        prex = 's' + (str(self.subject) if self.subject > 9 else '0' + str(self.subject)) + '/s' + (
            str(self.subject) if self.subject > 9 else '0' + str(self.subject)) + '_trial' + (
                   str(trial) if trial > 9 else '0' + str(trial)) + '/s' + (
                   str(self.subject) if self.subject > 9 else '0' + str(self.subject)) + '_trial' + (
                   str(trial) if trial > 9 else '0' + str(trial))
        transform = T.Compose([T.Resize((64, 64)),
                               T.ToTensor()])

        face_data = []
        for n in range(1, 6):
            # print(prex + f'_{(segment - 1) * 5 + n}.png')
            try:
                img = Image.open(io.BytesIO(self.face_zip.read(prex + f'_{(segment - 1) * 5 + n}.png')))

                frame_array = transform(img)
                frame_array = frame_array.view(1, 3, 64, 64)
                face_data.append(frame_array)
            except:
                print(prex + f'_{(segment - 1) * 5 + n}.png' + " file not exists in location")
            # print(prex + f'_{(segment - 1) * 5 + n}.png')

        if len(face_data) == 0:
            return None, None

        while len(face_data) != 5:
            face_data.append(face_data[0])

        face_data = torch.cat(face_data, dim=0)


        bio_data = torch.tensor(
            np.load(io.BytesIO(self.bio_zip.read(f's{self.subject}/{self.subject}_{trial}_{segment}.npy')))).float()

        if self.modal == 'face':
            data = face_data
        elif self.modal == 'eeg':
            data = bio_data[:32]
        

        valence = 0 if self.labels[(self.labels['Participant_id']==self.subject) & (self.labels['Trial']==trial)]['Valence'].iloc[0] < 5 else 1
        arousal = 0 if self.labels[(self.labels['Participant_id']==self.subject) & (self.labels['Trial']==trial)]['Arousal'].iloc[0] < 5 else 1
       
        if self.label == 'valence':
            return data, valence
        else:
            return data, arousal

    def __len__(self):
        return len(self.indices)

In [214]:
def train_model(lr = 0.001, batch_size = 32, epochs = 50, samples = 74040):

    train_eeg = Deap(kind = "train", samples = samples)
    test_eeg = Deap(kind = "test", samples = samples)

    train_loader = DataLoader(train_eeg, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(test_eeg, batch_size=batch_size, shuffle=True)

    device = torch.device('cuda')
    

    model = CNN_Model()
    
    criterion = torch.nn.BCELoss()
    lr = lr 
    optimizer = torch.optim.Adam(model.parameters(), lr = lr)

    loss_meter = meter.AverageValueMeter()
    
    #  creating a file results 
    try : 
        os.mkdir("./results")
    except Exception as E:
        print("File results made")
    best_accuracy = 0

    file_name = "./results/result.txt"
    for epoch in range(epochs):
        pred_label = []
        true_label = []

        loss_meter.reset()
        for ii, (data, label) in enumerate(train_loader):
        
            input_ = data.float().to(device)
            label = label.float().to(device)

            optimizer.zero_grad()
            
            pred = model(input_).float()
            pred = pred.squeeze().float()
            # If there's only one element, enclose it in brackets, to be tensor of one-dimension
            if pred.dim() == 0:
                pred = pred.unsqueeze(0)

            # print(pred, label, ii)
            loss = criterion(pred, label)
            loss.backward()
            optimizer.step()

            # meters update
            loss_meter.add(loss.item())

            pred = (pred >= 0.5).float().to(device).data
            pred_label.append(pred)
            true_label.append(label)

        pred_label = torch.cat(pred_label,0)
        true_label = torch.cat(true_label,0)

        print(epoch, torch.sum(pred_label == true_label), true_label.size(0))
        train_accuracy = torch.sum(pred_label == true_label).type(torch.FloatTensor) / true_label.size(0)
        out_put('Epoch: ' + 'train' + str(epoch) + '| train accuracy: ' + str(train_accuracy.item()), file_name)

        val_accuracy = val(model, val_loader, use_gpu = True)
        out_put('Epoch: ' + 'train' + str(epoch) + '| train loss: ' + str(loss_meter.value()[0]) +
              '| val accuracy: ' + str(val_accuracy.item()), file_name)

        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_epoch = epoch
            model.save(f"{file_name}_best.pth")

    model.save(f'{file_name}.pth')

    perf = f"best accuracy is {best_accuracy} in epoch {best_epoch}" + "\n"
    out_put(perf,file_name)

    return best_accuracy

In [215]:
x = train_model() 

File results made
0 tensor(33729, device='cuda:0') 59232
tensor(8488, device='cuda:0') 14808
1 tensor(35468, device='cuda:0') 59232
tensor(9091, device='cuda:0') 14808
2 tensor(37377, device='cuda:0') 59232
tensor(9844, device='cuda:0') 14808
3 tensor(39762, device='cuda:0') 59232
tensor(10403, device='cuda:0') 14808
4 tensor(41315, device='cuda:0') 59232
tensor(10713, device='cuda:0') 14808
5 tensor(42676, device='cuda:0') 59232
tensor(11028, device='cuda:0') 14808
6 tensor(43681, device='cuda:0') 59232
tensor(11244, device='cuda:0') 14808
7 tensor(44468, device='cuda:0') 59232
tensor(11478, device='cuda:0') 14808
8 tensor(45422, device='cuda:0') 59232
tensor(11797, device='cuda:0') 14808
9 tensor(46210, device='cuda:0') 59232
tensor(11915, device='cuda:0') 14808
10 tensor(46719, device='cuda:0') 59232
tensor(12066, device='cuda:0') 14808
11 tensor(47401, device='cuda:0') 59232
tensor(12217, device='cuda:0') 14808
12 tensor(47717, device='cuda:0') 59232
tensor(12267, device='cuda:0') 

In [203]:
@torch.no_grad()
def val(model, dataloader, use_gpu):
    model.eval()
    if use_gpu:
      device = torch.device('cuda')
    else:
      device = torch.device('cpu')

    pred_label = []
    true_label = []

    for ii, (data, label) in enumerate(dataloader):
        
        input_ = data.float().to(device)
        label = label.to(device)
        pred = model(input_).float()

        pred = (pred >= 0.5).float().to(device).data
        pred = pred.squeeze().float()
        
        # If there's only one element, enclose it in brackets, to be tensor of one-dimension
        if pred.dim() == 0:
            pred = pred.unsqueeze(0)
        pred_label.append(pred)
        true_label.append(label)

    # print(pred_label, len(pred_label))
    # print(true_label,len(true_label))
    pred_label = torch.cat(pred_label, 0)
    true_label = torch.cat(true_label, 0)
    
    print(torch.sum(pred_label == true_label), true_label.size(0))
    val_accuracy = torch.sum(pred_label == true_label).type(torch.FloatTensor) / true_label.size(0)

    model.train()

    return val_accuracy

In [137]:
def out_put(string, verbose):
    '''
    Help function for verbose,
    output the string to destination path

    Parameters
    ----------
    string  :str,  the string to output
    verbose :str, the path to store the output
    '''
    with open(f"{verbose}.txt", "a") as f:
        f.write(string + "\n")